<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. <br> 
   Editado por Equipo Docente IIC2233 2018-1, 2018-2, 2019-1 y 2019-2, y extendido con material creado en 2017-2 por Hugo Navarrete e Ignacio Acevedo.</font>
</p>

# Eventos y señales

Recordemos que las interfaces gráficas son aplicaciones construidas con una **arquitectura basada en manejo de eventos**. Esto significa que la aplicación detecta eventos generados por el usuario o por otras partes de la misma aplicación, y los procesa con el elemento apropiado. 

Una aplicación construida con PyQt empieza a detectar eventos una vez que ha entrado en su *mainloop*, y esto ocurre luego de llamar al método ```exec_()``` de `QApplication`. La siguiente figura muestra una comparación, mediante diagramas de flujo, entre un programa con una estructura lineal como los que habíamos construido en las semanas anteriores, y un programa con uso de GUI basada en manejo de eventos.

![](img/GUI-flowchart.png)

En este modelo existen 3 elementos fundamentales:

- **La fuente del evento**: Corresponde al objeto que genera el cambio de estado o que genera el evento
- **El objeto evento**: Es el objeto que encapsula el cambio de estado mediante el evento. 
- **El objeto destino:** Equivale al objeto al que se le desea notificar del cambio de estado

Bajo este modelamiento, la **fuente del evento** delega la tarea de manejar el evento al **objeto destino** pasándole el **objeto evento**. Para implementar este modelo, PyQt utiliza un mecanismo de ***signal*** y ***slot*** que le permite manejar los eventos y permitir comunicación entre *widgets*. De esta forma, cuando un evento ocurre, el objeto que es activado emite una señal (*signal*) a la ranura (*slot*) correspondiente, donde un *slot* puede ser cualquier tipo de elemento llamable (*callable*) en Python.

En el siguiente ejemplo, extenderemos el programa anterior que mostraba botones dispuestos en grilla que simulaban una calculadora, para generar una llamada a la función `boton_clickeado` cada vez que se presiona alguno de los botones. Esto se logra conectando el evento que enviará la señal con el *slot* que la recibe. En el caso de los botones, generalmente el método corresponde al evento `clicked`. Mediante el método `connect()` se establece la comunicación entre el evento y el *slot*. Este método recibe una función *llamable* en Python, (recordemos no agregar los `()`). El  ejemplo solo muestra la clase `MiVentana`.

In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel,
                             QLineEdit, QGridLayout, QVBoxLayout)

class MiVentana(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self):
        self.label1 = QLabel('Status:', self)
        self.grilla = QGridLayout()

        valores = ['1', '2', '3',
                   '4', '5', '6',
                   '7', '8', '9',
                   '0', 'CE', 'C']

        posiciones = [(i, j) for i in range(4) for j in range(3)]
        
        for posicion, valor in zip(posiciones, valores):
            boton = QPushButton(valor, self)

            """
            Aquí conectamos el evento clicked() de cada boton con el slot
            correspondiente. En este ejemplo todos los botones usan el
            mismo slot (self.boton_clickeado).
            """
            boton.clicked.connect(self.boton_clickeado)

            self.grilla.addWidget(boton, *posicion)

        vbox = QVBoxLayout()
        vbox.addWidget(self.label1)
        vbox.addLayout(self.grilla)
        self.setLayout(vbox)

        self.move(300, 150)
        self.setWindowTitle('Calculator')

    def boton_clickeado(self):
        """
        Esta función se ejecuta cada vez que uno de los botones de la grilla
        es presionado. Cada vez que el botón genera el evento, la función inspecciona
        cual botón fue presionado y recupera la posición en que se encuentra en
        la grilla.
        """

        # Sender retorna el objeto que fue clickeado.
        # Ahora, la variable boton referencia una instancia de QPushButton
        boton = self.sender()

        # Obtenemos el identificador del elemento en la grilla
        idx = self.grilla.indexOf(boton)

        # Con el identificador obtenemos la posición del ítem en la grilla
        posicion = self.grilla.getItemPosition(idx)

        # Actualizamos label1
        self.label1.setText(f'Status: Presionado boton {idx}, en fila/columna: {posicion[:2]}.')

if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    form.show()
    sys.exit(app.exec_())

## Obtener al emisor de la señal: `sender`

En ocasiones, como en la función `boton_clickeado()` del código anterior, es necesario conocer cuál de los objetos del formulario envió una señal. El método `sender()` retorna el objeto que generó el evento. El siguiente ejemplo utiliza `sender()` para obtener una referencia al objeto que generó el evento y mostrar cuál objeto generó el evento. 

En el mismo ejemplo, puedes notar que se usa la clase `QCoreApplication`, de `QtCore`, la que permite obtener una referencia al objeto `QApplication` que está en ejecución. Esto se utiliza para forzar una salida al hacer *click* en el *slot* del tercer botón.

In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel, QHBoxLayout, QVBoxLayout)
from PyQt5.QtCore import QCoreApplication


class MiVentana(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self):
        self.label1 = QLabel('Status: -', self)

        """
        El evento de cada botón es conectado con su slot. En este caso es 
        el mismo método boton_callback().
        """
        self.boton1 = QPushButton('&Boton 1', self)
        self.boton1.resize(self.boton1.sizeHint())
        self.boton1.clicked.connect(self.boton_clickeado)

        self.boton2 = QPushButton('&Boton 2', self)
        self.boton2.resize(self.boton2.sizeHint())
        self.boton2.clicked.connect(self.boton_clickeado)

        self.boton3 = QPushButton('&Salir', self)
        self.boton3.resize(self.boton3.sizeHint())
        self.boton3.clicked.connect(QCoreApplication.instance().quit)

        hbox = QHBoxLayout()
        hbox.addStretch(1)
        hbox.addWidget(self.boton1)
        hbox.addWidget(self.boton2)
        hbox.addWidget(self.boton3)
        hbox.addStretch(1)

        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addWidget(self.label1)
        vbox.addLayout(hbox)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Agregamos todos los elementos al formulario
        self.setGeometry(200, 100, 300, 200)
        self.setWindowTitle('Sender')

    def boton_clickeado(self):
        # Esta función registra el objeto que envía la señal del evento
        # y lo refleja mediante el método sender() en label3.
        sender = self.sender()
        self.label1.setText(f'Status: presionado botón {sender.text()}')
        self.label1.resize(self.label1.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    form.show()
    sys.exit(app.exec_())


## Eventos de *mouse*

### *Press Event*

No solo los botones tienen eventos de interacción con el usuario implementados, todo *widget* tiene distintos gatillos de eventos que se pueden conectar a *slots*. Por ejemplo, la clase `QWidget` incluye el método `mousePressEvent`, que se hace cargo del comportamiento del programa ante presionar el *mouse* sobre el *widget* que lo implementa. Por defecto, este método no hace nada, pero siempre podemos hacer *override* de el en nuestros propios *widgets*. Similar a como `QPushButton` tenía una palabra reservada para el evento `clicked` para el cual podía vincular un método o función, `mousePressEvent` refleja el evento genérico de un clic para cualquier *widget*. Pero, la diferencia es que no es necesario realizar la conexión evento-método (`.clicked.connect(...)`), para este tipo de eventos (y los siguientes ejemplos) la conexión ya está definida y es solo necesario sobreescribir el método correspondiente.

Es decir, al sobrescribir el método `mousePressEvent` de un `QWidget` se está definiendo el código que se ejecutará una vez que se haga clic sobre ese *widget*.

En el siguiente ejemplo, hay dos *labels* de colores dentro del *widget*. Cuando se haga clic sobre él, dependiendo de la posición presionada, se ocultará o mostrará el *label* correspondiente.

In [ ]:
import sys
import os
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel)
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import QRect


class MiVentana(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self):
        self.setGeometry(300, 100, 225, 450)
        self.setMaximumHeight(450)
        self.setMaximumWidth(225)
        self.setWindowTitle('mousePressEvent')
        
        self.mostrar_azul = True
        self.label_azul = QLabel('AZUL', self)
        self.label_azul.move(0, 0)
        self.label_azul.setGeometry(QRect(0, 0, 225, 225))  # (x, y, height, width)
        ruta_imagen_azul = os.path.join('img' ,'colors', 'azul.png')
        self.pixmap_azul = QPixmap(ruta_imagen_azul)
        self.label_azul.setPixmap(self.pixmap_azul)
        self.label_azul.show()
        
        self.mostrar_verde = True
        self.label_verde = QLabel('VERDE', self)
        self.label_verde.move(0, 0)
        self.label_verde.setGeometry(QRect(0, 225, 225, 225))
        ruta_imagen_verde = os.path.join('img' ,'colors', 'verde.png')
        self.pixmap_verde = QPixmap(ruta_imagen_verde)  
        self.label_verde.setPixmap(self.pixmap_verde)       
        self.label_verde.show()
        
        self.show()

    def mousePressEvent(self, event):
        if event.y() <= 225: # Este es el alto del label
            if self.mostrar_azul:
                self.label_azul.hide()
            else:
                self.label_azul.show()
            self.mostrar_azul = not self.mostrar_azul
        else:
            if self.mostrar_verde:
                self.label_verde.hide()
            else:
                self.label_verde.show()
            self.mostrar_verde = not self.mostrar_verde


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec_())

### *Move Event*

Otro aspecto en que nuestro programa puede estar pendiente del usuario, es en revisar la posición en la que se encuentra el *mouse* **cuando este está sobre el *widget* (sin necesariamente presionar)**. 

En el siguiente ejemplo, vamos a ver la posición global del *mouse* mediante el evento `mouseMoveEvent`. Para esto, vamos a activar el *tracking* del *mouse* en el `widget` y `label` que queramos seguir. En el ejemplo, solo lo activamos en el *label* azul; el *label* verde no mostrará la posición del *mouse* en consola. 

In [ ]:
import sys
import os
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import QRect


class MiVentana(QWidget):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self):
        self.setGeometry(300, 100, 225, 450)
        self.setMaximumHeight(450)
        self.setMaximumWidth(225)
        self.setWindowTitle('Move Event')
        
        # Creamos el label
        self.label_azul = QLabel('AZUL', self)
        self.label_azul.move(0, 0)
        self.label_azul.setGeometry(QRect(0, 0, 225, 225))  # (x, y, height, width)
        
        self.label_verde = QLabel('VERDE', self)
        self.label_verde.move(0, 0)
        self.label_verde.setGeometry(QRect(0, 225, 225, 225))
        
        ruta_imagen_azul = os.path.join('img' ,'colors', 'azul.png')
        self.pixmap_azul = QPixmap(ruta_imagen_azul) # Creamos el pixmap
        ruta_imagen_verde = os.path.join('img' ,'colors', 'verde.png')
        self.pixmap_verde = QPixmap(ruta_imagen_verde)
        
        self.label_azul.setPixmap(self.pixmap_azul) # Asignamos el pixmap
        self.label_verde.setPixmap(self.pixmap_verde)
        
        self.setMouseTracking(True) # Activamos el tracking en nuestra ventana
        self.label_azul.setMouseTracking(True)
        self.label_azul.show()
        self.label_verde.show()
        self.show()

    def mouseMoveEvent(self, event):
        objeto_posicion = event.pos()
        print(objeto_posicion.x(), objeto_posicion.y()) 


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec_())

Existen muchos más eventos relacionados con el mouse que puedes revisar explorando la documentación de PyQt en la web.

## Eventos del teclado

Otra de las maneras más comunes en que una interfaz gráfica genera eventos por parte del usuario,
es a través del teclado. Cuando un usuario **presiona una tecla** ocurren eventos para los que muchas veces queremos definir un comportamiento. La clase `QWidget` incluye el método `keyPressEvent`, que se hacen cargo del comportamiento del programa ante la presión de una tecla. Por defecto, este método no hace nada, pero siempre podemos hacer *override* de el en nuestros propios *widgets*.

In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self):
        self.etiqueta = QLabel('Etiqueta', self)
        self.etiqueta.move(20, 10)
        self.resize(self.etiqueta.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Teclado')
        self.show()

    def keyPressEvent(self, event):
        """
        Este método maneja el evento que se produce al presionar las teclas.
        """
        self.etiqueta.setText(f'Presionaron la tecla: {event.text()} de código: {event.key()}')
        self.etiqueta.resize(self.etiqueta.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    ex = MiVentana()
    sys.exit(app.exec_())


## Generar señales personalizadas

PyQt permite definir señales personalizadas por el programador, además de los predeterminados sobre el *mouse* y teclado. Para esto se debe crear un objeto que alojará (como atributo) la nueva señal, que debe ser subclase de `QtCore.QObject`. Dentro de la clase del objeto se crea la nueva señal como una instancia de `QtCore.pyqtSignal`. Finalmente, en los *widgets* involucrados se deben recibir las señales y conectar las funciones que manejan la señal. 

- Mediante el método `emit` de `pyqtSignal` se emite la señal (indicando que ocurrió un evento).
- Mediante el método `connect`, también de `pyqtSignal`, se define la función o método a ejecutar cuando la señal es emitida.

El siguiente ejemplo muestra cómo generar una nueva señal entre dos ventanas separadas. La señal se activa al presionar alguna parte la ventana izquierda, y su efecto es editar el contenido de la ventana derecha. 

La señal se crea como un atributo de clase dentro de la clase `MiSenal`. Se instancia dicha clase, y se le pasa como argumento a ambas ventanas. `VentanaPresionable` recibe la señal y es quien la **emite el evento** cada vez que se presiona la ventana, mientras que `VentanaQueSeEdita` recibe la señal y la **conecta** a uno de sus métodos, y por lo tanto **recibe el evento** que editará el contenido cada vez que se emita la señal.

In [ ]:
import sys
from PyQt5.QtCore import (QObject, pyqtSignal)
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)


class MiSenal(QObject):
    """
    Esta clase contiene la señal que permite la comunicación entre
    elementos de la GUI.
    """
    senal = pyqtSignal()


class VentanaPresionable(QWidget):

    def __init__(self, senal_escribir):
        super().__init__()

        self.senal_escribir = senal_escribir

        self.inicializa_gui()

    def inicializa_gui(self):
        self.etiqueta = QLabel('Presiona esta ventana', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Emite señal')
        self.show()

    def mousePressEvent(self, event):
        # Al ejecutar la siguiente línea, se emite la señal,
        # y los métodos conectados se llamarán automáticamente.
        self.senal_escribir.emit()

class VentanaQueSeEdita(QWidget):

    def __init__(self, senal_editar):
        super().__init__()

        self.senal_editar = senal_editar
        # Conectamos el método encargado de ejecutar la tarea
        self.senal_editar.connect(self.edita_etiqueta)

        self.inicializa_gui()

    def inicializa_gui(self):

        self.etiqueta = QLabel('', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.setGeometry(700, 300, 290, 150)
        self.setWindowTitle('Recibe señal')
        self.show()

    def edita_etiqueta(self):
        self.etiqueta.setText('¡Oh! Alguien ha presionado el mouse')
        self.etiqueta.resize(self.etiqueta.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    senal = MiSenal()
    ventana_1 = VentanaPresionable(senal.senal)
    ventana_2 = VentanaQueSeEdita(senal.senal)
    sys.exit(app.exec_())

#### (Abre paréntesis...

1. ¿Por qué se encapsula la instancia `pyqtSignal` dentro de una clase? ¿No se puede instanciar directamente y trabajar con el objeto `pyqtSignal`?
2. ¿Por qué se define como un atributo de clase y no de instancia?
3. ¿Esto último significa que todas las instancias de la clase `MiSenal` compartirán la misma señal?

¡Muy buenas preguntas! Las tres prácticamente se responden de la misma forma: la razón de fondo es la implementación interna de PyQt que requiere estas condiciones. Ahora, específicamente:

1. Los `pyqtSignal` deben estar encapsulados por objetos que hereden de `QObject`. Nota que `MiSenal` hereda de esta clase. Se pueden instanciar senales fuera de una clase, pero esta no será capaz de conectarse a funciones o ser emitidas por alguna entidad. Lo que sí se puede, es definir como atributo de clase directamente en la clase de un *widget* como una ventana, ya que todo `QWidget` es a su vez un `QObject`.
2. Las instancias de `pyqtSignal` deben ser definidas como atributo de clase y no de instancia por diseño interno de PyQt. Puedes hacer la prueba e instanciarla como un atributo de instancia, y verás que esta no será capaz de conectarse a llamables o de emitirse.
3. Hemos visto que el comportamiento de un atributo de clase es distinto para un atributo de instancia para objetos mutables. En general, si tenemos un atributo de clase mutable (como una lista), si cualquier instancia modifica este atributo, todas las instancias tienen acceso a este atributo y ven el efecto. Entonces, ¿todas las instancias de `MiSenal` compartirían la señal `senal`? Soprendentemente, la respuesta es **no**. Esto se debe a implementación interna de PyQt que logra que a pesar de que se define a nivel de atributo de clase, para cada instancia la señal creada como atributo es única y distinta. Entonces, a pesar de que se defina a ese nivel, podemos estar tranquilos en que cada instancia tendrá su propia señal única.

#### ...cierra paréntesis)

### Emisión de eventos con información

En nuestro primer ejemplo básico de señal personalizada, se emitió y recibió un evento que simplemente ocurría y realizaba un efecto. Este no transmitía más información ni datos que describieran el evento que ocurrió. `pyqtSignal` permite una forma de que al momento de emitir el evento (`emit`) se envíe también información a la función o método conectado y lo reciba como un argumento. Al instanciar la señal `pyqtSignal`, este recibe como argumentos los tipos de objetos que es capaz de enviar mediante `emit` y recibirán los objetos llamables como argumentos. En el siguiente ejemplo se expande el anterior con tres señales separadas que emiten distinta información:

In [ ]:
import sys
from PyQt5.QtCore import (QObject, pyqtSignal)
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)


class MisSenales(QObject):
    """
    Esta clase contiene las señales que permite la comunicación entre
    elementos de la GUI.
    """
    senal_simple = pyqtSignal() # Señal simple
    senal_texto = pyqtSignal(str) # Señal que permite enviar texto
    senal_coordenadas = pyqtSignal(int, int) # Señal que permite enviar dos ints


class VentanaPresionable(QWidget):

    def __init__(self, senal_simple, senal_texto, senal_coor):
        super().__init__()
        # Se guarda referencia a todas las señales que puede emitir esta ventana
        self.senal_simple = senal_simple
        self.senal_texto = senal_texto
        self.senal_coor = senal_coor

        self.inicializa_gui()

    def inicializa_gui(self):
        self.etiqueta = QLabel('Texto etiqueta', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Emite señal')
        self.setMouseTracking(True)
        self.show()

    def mousePressEvent(self, event):
        # Se emite la señal simple, sin argumento
        self.senal_simple.emit()
        # Se emite la señal que permite envir un str
        # Se envia el contenido de la etiqueta de la ventana
        self.senal_texto.emit(self.etiqueta.text())

    def mouseMoveEvent(self, event):
        # Se emite la señal que permite enviar dos ints, enviamos la posición del mouse
        self.senal_coor.emit(event.pos().x(), event.pos().y())

class VentanaQueSeEdita(QWidget):

    def __init__(self, senal_simple, senal_texto, senal_coor):
        super().__init__()

        self.senal_simple = senal_simple
        self.senal_texto = senal_texto
        self.senal_coor = senal_coor
        # Conectamos los métodos de respuesta de cada señal
        self.senal_simple.connect(self.edita_etiqueta_1)
        self.senal_texto.connect(self.edita_etiqueta_2)
        self.senal_coor.connect(self.edita_etiqueta_3)

        self.inicializa_gui()

    def inicializa_gui(self):

        self.etiqueta_1 = QLabel('', self)
        self.etiqueta_1.move(20, 10)
        self.etiqueta_1.resize(self.etiqueta_1.sizeHint())

        self.etiqueta_2 = QLabel('', self)
        self.etiqueta_2.move(20, 40)
        self.etiqueta_2.resize(self.etiqueta_2.sizeHint())

        self.etiqueta_3 = QLabel('', self)
        self.etiqueta_3.move(20, 70)
        self.etiqueta_3.resize(self.etiqueta_3.sizeHint())

        self.setGeometry(700, 300, 290, 150)
        self.setWindowTitle('Recibe señal')
        self.show()
    
    def edita_etiqueta_1(self):
        # Este método no tiene argumentos, ya que es una señal simple
        self.etiqueta_1.setText('¡Oh! Alguien ha presionado el mouse')
        self.etiqueta_1.resize(self.etiqueta_1.sizeHint())

    def edita_etiqueta_2(self, texto):
        # Este método tiene un argumento, el str que se espera del evento conectado
        self.etiqueta_2.setText(f'Recibí del evento: {texto}')
        self.etiqueta_2.resize(self.etiqueta_2.sizeHint())

    def edita_etiqueta_3(self, x, y):
        # Este método tiene dos argumentos, los ints que se espera del evento conectado
        self.etiqueta_3.setText(f'Recibí posiciones: {x}, {y}')
        self.etiqueta_3.resize(self.etiqueta_3.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    senales = MisSenales()
    ventana_1 = VentanaPresionable(senales.senal_simple, senales.senal_texto, senales.senal_coordenadas)
    ventana_2 = VentanaQueSeEdita(senales.senal_simple, senales.senal_texto, senales.senal_coordenadas)
    sys.exit(app.exec_())

### Señales personalizadas y *threads*

Otro uso de las señales personalizadas es en una aplicación *multithreaded*. El siguiente ejemplo muestra cómo crear una señal que controla la acción del *thread* sobre el formulario mediante el método ``actualizar_labels()``. El *thread* por su parte, recibe como parámetro la señal y emite mensajes.

In [ ]:
import sys
from threading import Thread
from time import sleep

from PyQt5.QtCore import pyqtSignal
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QHBoxLayout,
                             QVBoxLayout, QPushButton)


class MiThread(Thread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, senal_actualizar, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.senal_actualizar = senal_actualizar

    def run(self):
        for i in range(10):
            sleep(0.5)
            self.senal_actualizar.emit(str(i))
        self.senal_actualizar.emit('Status: thread terminado')


class MiVentana(QWidget):

    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.thread = None
        # Conectamos la señal del thread al método que maneja
        self.senal_thread.connect(self.actualizar_labels)

        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        self.label = QLabel('Status: esperando thread', self)
        self.boton = QPushButton('Ejecutar Thread', self)
        self.boton.clicked.connect(self.ejecutar_threads)

        hbox1 = QHBoxLayout()
        hbox1.addStretch(1)
        hbox1.addWidget(self.label)
        hbox1.addStretch(1)

        hbox2 = QHBoxLayout()
        hbox2.addStretch(1)
        hbox2.addWidget(self.boton)
        hbox2.addStretch(1)

        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addLayout(hbox1)
        vbox.addStretch(1)
        vbox.addLayout(hbox2)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle('Ejemplo threads')
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        if self.thread is None or not self.thread.is_alive():
            self.thread = MiThread(self.senal_thread)
            self.thread.start()

    def actualizar_labels(self, evento):
        """
        Este método actualiza el label según los datos enviados desde el
        thread através del objeto evento. Para este ejemplo, el método
        recibe el evento, pero podría también no recibir nada.
        """
        self.label.setText(evento)


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec_())